In [ ]:
import os

# カレントディレクトリをmagentaに
os.chdir(r"C:\Users\arkw\GitHub\magenta")
print(os.getcwd())

In [ ]:
def calculate_c_diatonic(notes):
    """Cのダイアトニック・スケールに該当する音符の割合を計算
    引数:
        notes: pretty_midi.Noteオブジェクトのリスト
    戻り値:
        Cのダイアトニック・スケールに該当する音符の割合
    """
    c_diatonic_scale = {0, 2, 4, 5, 7, 9, 11}
    diatonic_notes = [(note.pitch % 12) in c_diatonic_scale for note in notes]
    return sum(diatonic_notes) / len(notes) if notes else 0

def calculate_note_density(notes, tempo):
    """ノート密度の計算"""
    second_per_beat = 60 / tempo # 一拍あたりの秒数
    note_steps = notes[-1].end / second_per_beat * 4 # 16分音符のステップ数に変換
    return len(notes) / note_steps if note_steps else 0

def calculate_average_interval(notes):
    """平均音程間隔の計算"""
    intervals = [abs(notes[i].pitch - notes[i-1].pitch) for i in range(1, len(notes))]
    return sum(intervals) / len(intervals) if intervals else 0

def calculate_syncopation(notes, tempo, syncopation_type):
    """全ての音符に対するシンコペーション音符の割合を計算する
    引数:
        notes: pretty_midi.Noteオブジェクトのリスト
        tempo: テンポ。qpmと同義
        syncopation_type: 8thか16th
    戻り値:
        シンコペーション音符の割合
    """
    syncopated_count = 0 # シンコペーション音符の数
    prev_note_start = 0 # 一つ前の音符の開始時間
    second_per_beat = 60 / tempo # 一拍あたりの秒数
    if syncopation_type == "8th":
        value_for_evaluation = 0.5
    elif syncopation_type == "16th":
        value_for_evaluation = 0.25

    for note in notes:
        is_position = (note.start / second_per_beat) % (second_per_beat * 2) == value_for_evaluation
        interval = (note.start - prev_note_start) / second_per_beat
        if(is_position and interval > value_for_evaluation):
            syncopated_count += 1
        prev_note_start = note.start
    return (syncopated_count / len(notes)) if notes else 0

In [ ]:
import pretty_midi

# 例外オブジェクトを作るためのクラスを定義
# 読み込んだMIDIファイルが条件に合わない場合に
# このクラスによって定義される例外が投げられる
class UnsupportedMidiFileException(Exception):
  "Unsupported MIDI File"

def analyze_midi(file_path):
    midi_data = pretty_midi.PrettyMIDI(file_path)
    _, tempi = midi_data.get_tempo_changes()
    tempo = int(tempi[0])
    time_signature = midi_data.time_signature_changes[0]
    if midi_data.instruments:
      instrument = midi_data.instruments[0]

      # 属性計算
      c_diatonic = calculate_c_diatonic(instrument.notes) # Cのダイアトニック・スケールに該当する音符の割合
      note_num = len(instrument.notes) # ノートの数
      note_density = calculate_note_density(instrument.notes, tempo) # ノート密度
      average_interval = calculate_average_interval(instrument.notes) # 平均音程間隔
      syncopation_16th = calculate_syncopation(instrument.notes, tempo, "16th") # 16分シンコペーション音符の割合
      syncopation_8th = calculate_syncopation(instrument.notes, tempo, "8th") # 8分シンコペーションノートの割合

      return (file_path, tempo, time_signature, 
              c_diatonic, note_num, note_density, average_interval, syncopation_16th, syncopation_8th)
    else:
       # 空の楽譜の場合はこう
       return (file_path, tempo, time_signature,
               0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [ ]:
import glob
import os

midi_path = r"C:\Users\arkw\GitHub\magenta\tmp\music_vae\generated\0704"
files = glob.glob(os.path.join(midi_path, '*.mid'))

In [ ]:
import csv

def write_to_csv(results, csv_path):
    # ディレクトリが存在しない場合は作成する
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
     # CSVファイルに結果を保存
    with open(csv_path, 'w', newline='') as csvfile:  # 書き込みモード
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(results)  # 結果を1行に書き込む

In [ ]:
import pandas as pd

def write_to_csv(results, csv_path):
    # DataFrameを作成してCSVファイルに結果を保存
    df = pd.DataFrame(results, columns=['file_path', 'tempo', 'time_signature', 'c_diatonic', 'note_num', 'note_density', 'average_interval', 'syncopation_16th', 'syncopation_8th'])
    df.to_csv(csv_path, index=False)

In [ ]:
import datetime

results = []
for file in files:
    results.append(analyze_midi(file))

now = datetime.datetime.now()
filename = 'tmp/result_' + now.strftime('%Y%m%d_%H%M%S') + '.csv'
write_to_csv(results, filename)

指定された条件を満たすMIDIファイルを絞り込み、別の場所に保存する

In [ ]:
import pandas as pd
import shutil
import os

# CSVファイルの読み込み
csv_file_path = 'tmp/result_20240529_150740.csv'
df = pd.read_csv(csv_file_path)

# 条件を満たす行をフィルタリング
filtered_df = df[(df['note_num'] >= 4) & (df['average_interval'] < 8)]

# 保存先ディレクトリの指定
destination_dir = r'tmp\music_vae\generated\adjective01\filtered'
os.makedirs(destination_dir, exist_ok=True)

# 条件を満たすMIDIファイルをコピー
for file_path in filtered_df['file_path']:
    shutil.copy(file_path, destination_dir)

print(f"Filtered MIDI files have been saved to {destination_dir}")

In [ ]:
import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'syncopation_8th'

csv_path = "tmp/result_20240708_145215.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist()

# dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [path.replace(".mid", ".npy") for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [path.replace(".mid", ".npy") for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)



In [ ]:
import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'c_diatonic'

csv_path = "magenta/tmp/result_20240708_145215.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist()

dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [dir + path for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [dir + path for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)



In [ ]:
import pandas as pd
import numpy as np

filtered_csv_path = "tmp/filtered20240610.csv"
df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df['c_diatonic'] >= df['c_diatonic'].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df['c_diatonic'] <= df['c_diatonic'].quantile(0.25)]['file_path'].tolist()

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 上位25%平均を求める
average_npy = np.mean(top_npy_arrays, axis=0)

print(average_npy)
